In [13]:
using LinearAlgebra
using Random
using StatsBase

In [4]:
# Model parameters.
γ = 1
δ = 0
Ω = 1
dΩ = 1e-4

# Initial state.
ψ0 = complex(float([1 ; 0]))

# Simulation parameters.
number_trajectories = 10
t_final = 50.
dt = 0.01;

In [5]:
# Operators.
σ = complex(float([0 1 ; 0 0]))
σz = complex(float([1 0 ; 0 -1]))

# Hamiltonian.
H = - δ * σ' * σ + Ω/2. * (σ' + σ)
# Jump operator.
M1 = sqrt(γ) * σ;

# Displaced operators for derivatives.
Hp = - δ * σ' * σ + (Ω + dΩ)/2. * (σ + σ')
Hm = - δ * σ' * σ + (Ω - dΩ)/2. * (σ + σ')
M1p = M1
M1m = M1

# Jumps operators have to be passed as a list.
M_l = [M1]
Mp_l = [M1p]
Mm_l = [M1m];

In [6]:
typeof(true)

Bool

In [53]:
function gillespie_fisher(H::Matrix{ComplexF64},
        Hp::Matrix{ComplexF64},
        Hm::Matrix{ComplexF64},
        M_l::Vector{Matrix{ComplexF64}},
        Mp_l::Vector{Matrix{ComplexF64}},
        Mm_l::Vector{Matrix{ComplexF64}},
        dθ::Float64,
        ψ0::Vector{ComplexF64},
        t_final::Float64,
        dt::Float64,
        number_trajectories::Int64,
        verbose::Bool=True)
    
    println("Gillespie-Fisher method in Julia - Welcome")
    println("=> Starting the pre-computation stage")
    t_range = 0.:dt:t_final
    
    # Constructs the overall jump operator.
    J = zero(M_l[1])
    for M in M_l
        J += M' * M
    end
    # Effective (non-Hermitian) Hamiltonian.
    He = H - 1im/2. * J
    
    # Constructs the no-jump evolution operators for all the relevant times.
    V = Matrix{ComplexF64}[] # List of the no-jump evolution operators.
    Qs = Matrix{ComplexF64}[] # List of the non-state-dependent part of the waiting time distribution.
    for t in t_range
        ev_op = exp(-1im * He * t)
        push!(V, ev_op)
        nsd_wtd = ev_op' * J * ev_op
        push!(Qs, nsd_wtd)
    end
    
    # Prints the matrix norm for the latest Qs.
    error = norm(last(Qs))
    println("-> Truncation error given by norm of latest Qs matrix: " * string(error))
    
    # Displaced version of the effective Hamiltonian.
    Jp = zero(Mp_l[1])
    for Mp in Mp_l
        Jp += Mp' * Mp
    end
    Hep = Hp - 1im/2. * Jp
    Jm = zero(Mm_l[1])
    for Mm in Mm_l
        Jm += Mm' * Mm
    end
    Hem = Hm - 1im/2. * Jm
    
    # Vector of the derivatives of the evolution operator wrt the parameter.
    Vdot = Matrix{ComplexF64}[]
    for t in t_range
        vd = (exp(-1im * Hep * t) - exp(-1im * Hem * t)) / (2 * dθ)
        push!(Vdot, vd)
    end
    # Derivatives of all the jump operators.
    Mdot = Matrix{ComplexF64}[]
    for n_M in eachindex(M_l)
        push!(Mdot, (Mp_l[n_M] - Mm_l[n_M]) / (2 * dθ))
    end
    # Precomputation of all the derivatives of MkV(t).
    Δ = Array{Matrix{ComplexF64}}[]
    for n_M in eachindex(M_l)
        fixed_M = Matrix{ComplexF64}[]
        for n_t in eachindex(t_range)
            obj = Mdot[n_M] * V[n_t] + M_l[n_M] * Vdot[n_t]
            push!(fixed_M, obj)
        end
        push!(Δ, fixed_M)
    end
    
    println("=> Starting the evolution cycle.")
    
    # List for the results.
    trajectories_results = Array{Dict{String, Any}}[]
    
    # Cycle over the trajectories.
    for trajectory in 1:number_trajectories
        println("-> Starting trajectory " * string(trajectory))
        
        # Initial state.
        ψ = ψ0
        # Absolute time.
        τ = 0
        # Initial ξ matrix (same size and type as the Hamiltonian).
        ξ = zero(H)
        
        results = Dict{String, Any}[]
        dict_initial = Dict("AbsTime" => 0,
            "TimeSinceLast" => 0,
            "JumpChannel" => nothing,
            "ξAfter" => ξ,
            "ψAfter" => ψ0)
        push!(results, dict_initial)
        
        while τ < t_final
            dict_jump = Dict()
            
            # Pass to density matrix formalism.
            ρ = ψ * ψ'
            
            # Compute the waiting time distribution, exploiting the pre-computed part.
            Ps = Float64[]
            for Q in Qs
                wtd = real(ψ' * Q * ψ)
                push!(Ps, wtd)
            end
            
            # Sample from the waiting time distribution.
            n_T = sample(1:length(t_range), Weights(Ps))
            
            # Increase the absolute time.
            τ += t_range[n_T]
            merge!(dict_jump, Dict("AbsTime" => τ, "TimeSinceLast" => t_range[n_T]))
            
            # Update the state.
            ψ = V[n_T] * ψ
            # Chooses where to jump.
            weights = Float64[]
            for M in M_l
                weight = real(ψ' * M' * M * ψ)
                push!(weights, weight)
            end
            n_jump = sample(1:length(M_l), Weights(weights))
            merge!(dict_jump, Dict("JumpChannel" => n_jump))
            # Update the state after the jump.
            ψ = M_l[n_jump] * ψ
            norm_state = norm(ψ)
            # Renormalize the state.
            ψ = ψ / norm_state
            
            if verbose
                println(string(dict_jump))
            end
            
            # Compute the ξ matrix.
            ξ = 1/(norm_state^2) * (M_l[n_jump] * V[n_T] * ξ * (V[n_T])' * (M_l[n_jump])')
            ξ += 1/(norm_state^2) * (Δ[n_jump][n_T] * ρ * (V[n_T])' * (M_l[n_jump])')
            ξ += 1/(norm_state^2) * (M_l[n_jump] * V[n_T] * ρ * (Δ[n_jump][n_T])')
            trξ2 = real(tr(ξ))^2
            merge!(dict_jump, Dict("ξAfter" => ξ, "ψAfter" => ψ, "trξ2" => trξ2))
            push!(results, dict_jump)
        end
        
        push!(trajectories_results, results)
    end
    
    return trajectories_results, V, Vdot, t_range
end

gillespie_fisher (generic function with 2 methods)

In [59]:
trajectories_results, V, Vdot, t_range = gillespie_fisher(H, Hp, Hm, M_l, Mp_l, Mm_l, dΩ, ψ0, t_final, dt, number_trajectories, false);

Gillespie-Fisher method in Julia - Welcome
=> Starting the pre-computation stage
-> Truncation error given by norm of latest Qs matrix: 1.9968814500797416e-11
=> Starting the evolution cycle.
-> Starting trajectory 1
-> Starting trajectory 2
-> Starting trajectory 3
-> Starting trajectory 4
-> Starting trajectory 5
-> Starting trajectory 6
-> Starting trajectory 7
-> Starting trajectory 8
-> Starting trajectory 9
-> Starting trajectory 10


In [64]:
function fisher_at_time_on_trajectory(t_range::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64},
        relevant_times::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64},
        V::Vector{Matrix{ComplexF64}},
        Vdot::Vector{Matrix{ComplexF64}},
        trajectory_data::Vector{Array{Dict{String, Any}}},
        ψ0::Vector{ComplexF64})
    
    v_fisher = Float64[]
end

fisher_at_time_on_trajectory (generic function with 1 method)

Vector{ComplexF64} (alias for Array{Complex{Float64}, 1})